In [ ]:
"""
Created on Mon Jan 20 11:50 2025

Compare the file currently used for IPSL-CM7 and the one I created to make them look similar

@author: Clara Burgard
"""

In [1]:
import xarray as xr
from multimelt.constants import *
import numpy as np
import matplotlib.pyplot as plt

In [2]:
%matplotlib qt5

QStandardPaths: error creating runtime directory '/run/user/2784' (Permission denied)


READ DATA

In [3]:
inputpath_raw='/data/cburgard/PREPARE_FORCING/PREPARE_PRESCRIBED_MELT/raw/'
inputpath_interim='/data/cburgard/PREPARE_FORCING/PREPARE_PRESCRIBED_MELT/interim/'
inputpath_raw2 = '/data/cburgard/PREPARE_FORCING/PREPARE_CAVITY_MASKS/raw/'


In [4]:
file_IPSL = xr.open_dataset(inputpath_raw + 'eORCA1.4.2_runoff-icb_DaiTrenberth_Depoorter.nc')
file_Davison = xr.open_dataset(inputpath_interim + 'Davison23_basal_melt_pattern_on_eORCA1.nc')
file_current = xr.open_dataset(inputpath_interim + 'masks_for_eORCA1_prescribedmeltinopencav.nc')

In [5]:
domain_cfg = xr.open_dataset(inputpath_raw2 + 'eORCA1.4.3_OpenSeas_OpenAllCav_ModStraights/eORCA1.4.3_OpenSeas_OpenAllCav_ModStraights_domain_cfg.nc')

In [6]:
cell_area = domain_cfg['e1t'] * domain_cfg['e2t']

In [17]:
melt_converted = file_current['melt_isf'] / yearinsec *10**(12) / cell_area 

In [18]:
melt_IPSL_Gt = file_IPSL['sornfisf'].isel(time_counter=0) * yearinsec * 10**(-12) * cell_area

In [ ]:
melt_IPSL_Gt.plot()

In [ ]:
melt_IPSL_Gt.where(melt_IPSL_Gt != 0).plot(vmin=-5,vmax=5,cmap = 'coolwarm')

In [31]:
file_current['melt_isf'].sum()

<xarray.DataArray 'melt_isf' ()>
array(1057.03720311)

In [ ]:
plt.figure()
(file_current['melt_isf']).plot(vmin=-5,vmax=5,cmap = 'coolwarm')

In [ ]:
(melt_converted - file_IPSL['sornfisf'].isel(time_counter=0))/file_IPSL['sornfisf'].isel(time_counter=0)).plot(vmax=1)

In [ ]:
(file_current['melt_isf'] - melt_IPSL_Gt).plot()

In [ ]:
melt_converted.plot()

In [19]:
modified_file_IPSL = file_IPSL.copy()

In [20]:
modified_file_IPSL['sornfisf'] = file_IPSL['sornfisf'] * 0 + melt_converted.where(np.isfinite(melt_converted),0).squeeze()
modified_file_IPSL['sornfisf'].attrs['long_name'] = 'runoff from antarctica iceshelves (Davison 2023)'

In [21]:
modified_file_IPSL['sodepmax_isf'] = file_IPSL['sodepmax_isf'] * 0 + file_current['z_max'].where(np.isfinite(file_current['z_max']),0).squeeze()
modified_file_IPSL['sodepmin_isf'] = file_IPSL['sodepmin_isf'] * 0 + file_current['z_min'].where(np.isfinite(file_current['z_min']),0).squeeze()

In [22]:
modified_file_IPSL.attrs['description2'] = 'south of 60 south : Davison Freshwater Discharge from basal front to grounding line iceshelves melting'
modified_file_IPSL.attrs['reference3'] = 'B.J. Davison, A.E. Hogg, N. Gourmelen, L. Jakob, J. Wuite, T. Nagler, C.A. Greene, J. Andreasen, M.E. Engdahl, 2023: Annual mass budget of Antarctic ice shelves from 1997 to 2021. Science Advances, doi:10.1126/sciadv.adi0186'
modified_file_IPSL.attrs['creation'] = 'Mon Jan 20 16:11 2025'

In [23]:
modified_file_IPSL.attrs['history5'] = 'modified from eORCA1.4.2_runoff-icb_DaiTrenberth_Depoorter.nc by replacing the ice shelf freshwater flux by values from Davison and prescribing the flux in large cavities to open them in NEMO (contact Clara Burgard)' 

In [ ]:
file_IPSL['sornfisf']

In [25]:
modified_file_IPSL.to_netcdf(inputpath_interim + 'eORCA1.4.2_runoff-icb_DaiTrenberth_Davison.nc')

In [ ]:
(modified_file_IPSL['sodepmax_isf'] - file_IPSL['sodepmax_isf']).isel(time_counter=0).plot()

In [ ]:
modified_file_IPSL['sodepmax_isf'].plot()

CHECKS

In [26]:
file_IPSL = xr.open_dataset(inputpath_raw + 'eORCA1.4.2_runoff-icb_DaiTrenberth_Depoorter.nc')
modified_file_IPSL = xr.open_dataset(inputpath_interim + 'eORCA1.4.2_runoff-icb_DaiTrenberth_Davison.nc')

In [29]:
((file_IPSL['sornfisf'] != 0).astype(int)*2 + (modified_file_IPSL['sornfisf'] != 0).astype(int)).isel(time_counter=0).plot()

In [13]:
modified_file_IPSL['sornfisf'].isel(time_counter=0).plot()

In [16]:
modified_file_IPSL['sodepmax_isf'].isel(time_counter=0).plot()